In [2]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python

In [3]:
bd.projects.set_current("ei38-teaching-25")

In [94]:
bd.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff

In [93]:
db = bd.Database('charcrete')
del bd.databases[db.name]

In [6]:
bd.projects.set_current("AS-group-project")

In [7]:
bd.databases

Databases dictionary with 4 object(s):
	EXIOBASE 3.8.1 2017 monetary
	EXIOBASE 3.8.1 2017 monetary biosphere
	biosphere3
	ei 3.8 cutoff

In [9]:
eidb = bd.Database("ei 3.8 cutoff")

In [33]:
forwarding_act = [act for act in eidb if 'forwarding' in act['name'] and act['location']=="RER"][0]

In [54]:
forwarding_act.as_dict()

{'comment': 'The inventory refers to 1 productive machine hour (PMS) - referring to the operational time of the machine including breaks up to 15 minutes - of a forwarder class II (10 - 12 t) with an average engine power of 110 kW and a total weight of 11,049 kg.\nA forwarder is a forestry vehicle that carries felled logs from the stump to a roadside landing. Unlike a skidder, a forwarder carries logs clear of the ground, which can reduce soil impacts but tends to limit the size of the logs it can move. Forwarders are typically employed together with harvesters in cut-to-length logging operations.\nImage: https://db3.ecoinvent.org/images/8840519f-6554-4a44-8589-fe5c5b02e11c\nGeography:  Emission profile according to European legislation\nTechnology:  Average european technology',
 'classifications': [('ISIC rev.4 ecoinvent', '0220:Logging'),
  ('CPC', '8614: Support and operation services to forestry and logging')],
 'activity type': 'ordinary transforming activity',
 'activity': '44df

In [40]:
chipping_act = [act for act in eidb if 'chipping' in act['name'] and 'mobile' in act['name'] and act['location']=="RER"][0]

In [45]:
chipping_act.key

('ei 3.8 cutoff', 'a4f64071a3a353f807b933a6ddfc1f81')

In [52]:
chip_transport = [
act for act in eidb if 
     "market for transport, freight, lorry, unspecified" in act['name'] 
    and act['location']=='RER'
    and not "generic" in act['name']
][0]

In [53]:
chip_transport.key

('ei 3.8 cutoff', '2b0c1793924e1bb3e821cbe15bee1a32')

In [58]:
elec = [
act for act in eidb if 
     "market for electricity, high voltage" in act['name'] 
    and act['location']=='SE'
][0]

In [59]:
elec.key

('ei 3.8 cutoff', 'b3e6b1396a7aad9719da6e6e4aef374a')

In [70]:
N20_act = [
act for act in bd.Database('biosphere3') if 
     'Dinitrogen' in act['name'] 
][0]

In [78]:
N20_act.items

<bound method Mapping.items of 'Dinitrogen monoxide' (kilogram, None, ('air', 'non-urban air or from high stacks'))>

In [81]:
CH4_act = [
act for act in bd.Database('biosphere3') if 
     'Methane, non-fossil' in act['name'] 
    and 'high stacks' in str(act['categories'])
][0]

In [82]:
CH4_act.key

('biosphere3', '57bdb443-d4a6-423d-8024-959b8261d02e')

In [95]:
db = bd.Database("charcrete_db")

In [96]:
charcrete_activities = {
    ("charcrete_db", '1'): {
        "name": "woodchip production",
        'unit': 'ton',
        'categories': ("comment", "category here"), #?
        'database': 'charcrete_db',
        "exchanges": [{
            "amount": 0.15739,
            "input": ('ei 3.8 cutoff', '39f73b6fa228fc6df946cc78891f1456'), #Forwarding
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "hour"},
                      {
            "amount": 0.047136,
            "input": ('ei 3.8 cutoff', 'a4f64071a3a353f807b933a6ddfc1f81'), #Chipping
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "hour"}
        ]
        },
        ("charcrete_db", '2'): {
        "name": "woodchip transport",
        'unit': 'ton',
        'categories': ("comment", "lorry, unspecified, RER - 200km"), #
        'database': 'charcrete_db',
        "exchanges": [{
            "amount": 200,
            "input": ('ei 3.8 cutoff', '2b0c1793924e1bb3e821cbe15bee1a32'), #transport
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "ton.km"},
        ]
        },
        ("charcrete_db", '3'): {
        "name": "pyrolysis",
        'unit': 'ton',
        'categories': ("comment", "For one ton of biomass, Sweden mix - HV"), #
        'database': 'charcrete_db',
        "exchanges": [{
            "amount": 0.010,
            "input": ('ei 3.8 cutoff', 'b3e6b1396a7aad9719da6e6e4aef374a'), #electricity HV
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "kWh"},
            {
            "amount": 0.108,
            "input": ('biosphere3', 'afd6d670-bbb0-4625-9730-04088a5b035e'), #N20
            "type": "biosphere",
            "uncertainty type":0,
            "unit=": "kilogram"},
                        {
            "amount": 0.198,
            "input": ('biosphere3', '57bdb443-d4a6-423d-8024-959b8261d02e'), #CH4
            "type": "biosphere",
            "uncertainty type":0,
            "unit=": "kilogram"},
        ]
        },
     ("charcrete_db", '4'): {
        "name": "biochar production",
        'unit': 'ton',
        'categories': ("comment", "category here"),
        'database': 'charcrete_db',
        "exchanges": [{
            "amount": 2.7, #amount is 1/0.37 as we get 370 kilo of biochar out of 1 ton woodchips
            "input": ("charcrete_db", "1"), #woodchip production
            "type": "technosphere",
            "uncertainty type":0,
            "unit": "ton"},
            {
            "amount": 2.7,
            "input": ("charcrete_db", "2"), #woodchip transport
            "type": "technosphere",
            "uncertainty type":0,
            "unit": "ton"},
            {
            "amount": 2.7,
            "input": ("charcrete_db", "3"), #pyrolysis
            "type": "technosphere",
            "uncertainty type":0,
            "unit": "ton"},
                      ]
        }
}

In [97]:
db.write(charcrete_activities)

Not able to determine geocollections for all datasets. This database is not ready for regionalization.
Title: Writing activities to SQLite3 database:
  Started: 10/26/2022 13:41:50
  Finished: 10/26/2022 13:41:50
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 0.37


In [100]:
act_t = db.random()
act_t

'pyrolysis' (ton, None, ('comment', 'For one ton of biomass, Sweden mix - HV'))

In [101]:
ipcc2013 = ('IPCC 2013', 'climate change', 'GWP 100a')

In [104]:
lca_w = bc.LCA({act_t:1},ipcc2013)

In [105]:
lca_w.lci()
lca_w.lcia()
lca_w.score

34.24181453257221